In [47]:
import re
import time
import random
import csv
from typing import Tuple, Dict, Optional, List

import requests
from bs4 import BeautifulSoup

import numpy as np

from tqdm import tqdm

import requests
import fitz  # PyMuPDF
import io

import lxml

# FAQ

In [35]:
url = "https://cbr.ru"

In [76]:
def extract_questions(dropdown_questions: Optional[Tuple], rubric_title: str, theme_name: str = "") -> Tuple[Dict]:
    items = []
    for dropdown_question in dropdown_questions:
        try:
            question = dropdown_question.find("h3") if dropdown_question.find("h3") else dropdown_question.find("h2")
            question = question.get_text().strip().replace("\xa0", " ")
            answer = dropdown_question.find("div", class_="additional-text-block").get_text().strip().replace("\xa0", " ")
            
            items.append({"title": rubric_title,"theme": theme_name, "question": question, "answer": answer})
        except Exception:
            print(f"Failed to parse question. Rubric: {rubric_title} \nTheme: {theme_name} \nDropdown_question: \n{dropdown_question}")

    return items

In [77]:
data = []
for rubric in rubrics:
    rubric_title = rubric.get_text().replace("\xa0", " ")
    rubric_url = url + rubric.get("href")
    print(rubric_title, rubric_url)

    res = requests.get(rubric_url)
    rubric_soup = BeautifulSoup(res.text)
    categories = rubric_soup.find_all("div", class_="dropdown dropdown_container")

    if not categories:
        dropdown_questions = rubric_soup.find_all("div", class_="dropdown question")
        items = extract_questions(
            dropdown_questions=dropdown_questions, rubric_title=rubric_title
        )
        data.extend(items)
        continue

    for category in categories:
        theme = category.find("div", class_="dropdown_title referenceable")
        if theme:
            theme_name = theme.find("h2").get_text().replace("\xa0", " ")
            dropdown_questions = category.find_all("div", class_="dropdown question")
            items = extract_questions(
                dropdown_questions=dropdown_questions, rubric_title=rubric_title, theme_name=theme_name
            )
            data.extend(items)
        else:
            time.sleep(random.randint(1, 2))

            theme = category.find("div", class_="dropdown_title-link").find("a", class_="dropdown_link referenceable")
            theme_url = url + theme.get("href")
            theme_name = theme.get_text().replace("\xa0", " ")

            theme_response = requests.get(theme_url)
            theme_soup = BeautifulSoup(theme_response.text)

            dropdown_questions = theme_soup.find_all("div", class_="dropdown question")
            items = extract_questions(
                dropdown_questions=dropdown_questions, rubric_title=rubric_title, theme_name=theme_name
            )
            data.extend(items)


        # print(theme_name)

        # answer = category.find()
        

Актуальные вопросы работы финансовой системы https://cbr.ru/faq/w_fin_sector/
Денежно-кредитная политика https://cbr.ru/faq/dkp/
Наличное денежное обращение https://cbr.ru/faq/ndo/
Национальная платежная система https://cbr.ru/faq/nps/
Банковский сектор https://cbr.ru/faq/bank_s/
Пенсионные фонды и коллективные инвестиции https://cbr.ru/faq/pens_k_inv/
Страхование https://cbr.ru/faq/insh/
Рынок ценных бумаг https://cbr.ru/faq/rcb/
Инфраструктура финансового рынка https://cbr.ru/faq/finm_infrastructure/
Микрофинансирование https://cbr.ru/faq/microfin/
Кредитные истории https://cbr.ru/faq/credit_h/
Развитие финансовых технологий https://cbr.ru/faq/fintekh/
Противодействие недобросовестным практикам https://cbr.ru/faq/pnp/
Платформа «Знай своего клиента»  https://cbr.ru/faq/client/
О Банке России https://cbr.ru/faqnew/br/
Информационная безопасность https://cbr.ru/faq/information_security/
Жалобы в Банк России  https://cbr.ru/faq/zhaloby-v-bank-rossii-/
Удостоверяющий центр Банка России h

In [78]:
data[:3]

[{'title': 'Актуальные вопросы работы финансовой системы',
  'theme': 'Меры поддержки мобилизованных и участников СВО',
  'question': 'Могу ли я, как участник СВО, переводить деньги без комиссии?',
  'answer': 'Банк России рекомендовал банкам снизить или отменить комиссии за переводы со счета участника СВО членам его семьи и за снятие наличных со счетов участников СВО и членов их семей.\nБанки самостоятельно устанавливают порядок реализации этих мер поддержки. Согласно рекомендациям Банка России, военнослужащий должен иметь возможность в доступной для него форме или через представителя сообщить банку о наличии оснований для сниженных комиссий и о том, на кого из членов семьи они должны распространяться.'},
 {'title': 'Актуальные вопросы работы финансовой системы',
  'theme': 'Меры поддержки мобилизованных и участников СВО',
  'question': 'Что такое кредитные каникулы для мобилизованных и участников СВО? По каким кредитам мобилизованные и другие участники СВО могут получить кредитные ка

In [79]:
len(data)

637

In [84]:
csv_file_path = "../data/questions.csv"
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=["title", "theme", "question", "answer"])
    
    csv_writer.writeheader()
    csv_writer.writerows(data)

print(f"Данные успешно сохранены в файл: {csv_file_path}")

Данные успешно сохранены в файл: ../data/questions.csv


In [ ]:
data = [{"theme": "", "question": "", "answer": ""}, ...]
data = {"some_theme": {"question": "", "answer": ""}}

# Documents

In [5]:
URL = f"https://cbr.ru/Crosscut/LawActs/Page/94917?Date.Time=Any&Page={i}"

In [19]:
def get_links():
    all_links = []
    for i in range(1, 297):
        response = requests.get(f"https://cbr.ru/Crosscut/LawActs/Page/94917?Date.Time=Any&Page={i}")
        soup = BeautifulSoup(response.text)

        cross_result = soup.find("div", class_="cross_results")

        links = ["https://cbr.ru" + item.get("href") for item in soup.find_all("a")]
        all_links.extend(links)
        if i % 10 == 0:
            print(f"{i} ссылок собрано!")
            time.sleep(random.randint(1, 3))
    return all_links
    

In [20]:
links = get_links()

10 ссылок собрано!
20 ссылок собрано!
30 ссылок собрано!
40 ссылок собрано!
50 ссылок собрано!
60 ссылок собрано!
70 ссылок собрано!
80 ссылок собрано!
90 ссылок собрано!
100 ссылок собрано!
110 ссылок собрано!
120 ссылок собрано!
130 ссылок собрано!
140 ссылок собрано!
150 ссылок собрано!
160 ссылок собрано!
170 ссылок собрано!
180 ссылок собрано!
190 ссылок собрано!
200 ссылок собрано!
210 ссылок собрано!
220 ссылок собрано!
230 ссылок собрано!
240 ссылок собрано!
250 ссылок собрано!
260 ссылок собрано!
270 ссылок собрано!
280 ссылок собрано!
290 ссылок собрано!


In [23]:
with open("../data/document_links.txt", "w") as file:
    for link in links:
        file.write(link + "\n")

In [24]:
len(links)

2957

In [29]:
string = "https://cbr.ru/Queries/UniDbQuery/File/90134/859"
pattern = r"https?://"

In [33]:
cbr_links = [item for item in links if len(re.findall(pattern=pattern, string=item)) < 2]

In [34]:
with open("../data/cbr_document_links.txt", "w") as file:
    for link in cbr_links:
        file.write(link + "\n")

In [35]:
len(cbr_links)

2693

In [41]:
import requests
import fitz  # PyMuPDF
import io

def download_pdf(url):
    response = requests.get(url)
    if response.status_code == 200:
        return io.BytesIO(response.content)
    else:
        return None

def extract_text_from_pdf_stream(pdf_stream):
    with fitz.open(stream=pdf_stream, filetype="pdf") as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text

# Ссылка на PDF-документ
def extract_documents(pdf_urls: List[str]):
    data = []
    for pdf_url in tqdm(pdf_urls):
        pdf_stream = download_pdf(pdf_url)

        if pdf_stream:
            text = extract_text_from_pdf_stream(pdf_stream)
            data.append({"doc_url": pdf_url, "text": text})
        else:
            print("Не удалось загрузить документ")
            continue
    
    return data

In [44]:
data = extract_documents(cbr_links[:20])

100%|██████████| 20/20 [00:02<00:00,  8.00it/s]


In [45]:
data[:3]

[{'doc_url': 'https://cbr.ru/Crosscut/LawActs/File/7682',
  'text': ' \n \n \n \n            ЦЕНТРАЛЬНЫЙ БАНК \n       РОССИЙСКОЙ ФЕДЕРАЦИИ \n                  (БАНК РОССИИ) \n             107016, Москва, ул. Неглинная, 12, к. В \n                                   www.cbr.ru \n           тел.: (499) 300-30-00, 8 (800) 300-30-00  \n \nОт [REGNUMDATESTAMP] \n \n   \n \nИнформационное письмо Банка России  \nоб отмене Информационного письма Банка \nРоссии от 9 марта 2016 года № ИН-05-35/11 \n \n \n \n \n \n \n \n \nКредитным организациям \n \nСтруктурным подразделениям \nБанка России  \n(по списку рассылки) \n \n \n \n \n \nБанк России сообщает, что в связи с пересмотром подходов  \nк установлению национальной антициклической надбавки1 с даты размещения \nнастоящего Информационного письма Банка России на официальном сайте \nБанка России в информационно-телекоммуникационной сети «Интернет» \nотменяется Информационное письмо Банка России от 9 марта 2016 года  \n№ ИН-05-35/11 «О методиках и 

In [46]:
csv_file_path = "../data/documents.csv"
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=["doc_url", "text"])
    
    csv_writer.writeheader()
    csv_writer.writerows(data)

print(f"Данные успешно сохранены в файл: {csv_file_path}")

Данные успешно сохранены в файл: ../data/documents.csv
